In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import ast
# from sodapy import Socrata
from collections import Counter
from datetime import datetime

In [ ]:
# read in a csv file
person = # download PSRC HTS person-level data
trips = # download PSRC HTS trip-level data
household = # download PSRC HTS household-level data
od = # insert dataset with associated origin-destiation lat/lng for household travel survey
home_loc = # insert dataset with associated home location lat/lng for household travel survey

# Join and clean data
- Identify which individuals have made restaurant trips
- gather ALL the trip data these individuals have made
- join O/D information to each

Inclduing weekend trips, we have 20,186 total restaurent trips made by 4045 individuals.

In [ ]:
# only restaurant trips
restaurant_trips = trips[(trips["origin_purpose"]=="Went to restaurant to eat/get take-out")
                            | (trips["dest_purpose"]=="Went to restaurant to eat/get take-out")]

# individuals who visited restaurants
rest_person_ids = restaurant_trips["person_dim_id"].unique()

In [ ]:
# remove na values
trips_clean = trips[(trips.arrival_time_string.notna()) & trips.depart_time_string.notna()]

In [ ]:
# gather all trip data for a person if they made a restaurant trip
selected_trips = trips_clean[trips_clean.person_dim_id.isin(rest_person_ids)]
selected_trips

In [ ]:
# function to manually calculate travel time so we don't have to drop rows
def calc_travel(row):
    date_str_arr = row.arrival_time_string[:-1]
    date_str_dep = row.depart_time_string[:-1]
    date_format = "Date: %Y-%m-%d %H:%M:%S.%f"

    # convert string to datetime object
    dt_object_arr = datetime.strptime(date_str_arr, date_format)
    dt_object_dep = datetime.strptime(date_str_dep, date_format)
    
    # calculate travel time
    delta_time = dt_object_arr - dt_object_dep
    travel_time_mins = delta_time.total_seconds()/60

    return travel_time_mins

In [ ]:
# self-calculate column for trip time
selected_trips["travel_time_calc"] = selected_trips.apply(calc_travel, axis=1)

# reduce dataset to only include trips with travel time < 300
selected_trips_clean = selected_trips[selected_trips.travel_time_calc < 120]

In [ ]:
# join o/d information
trips_od = pd.merge(selected_trips_clean, od, on=["trip_id", "household_id"], how="left")
# drop unecessary columns
trips_od = trips_od.drop(["person_id", "ObjectId"], axis = 1)
# drop 2021
trips_od = trips_od[trips_od["survey_year"] != 2021]
# trips_od = trips_od[(trips_od["daynum"] != 6) | (trips_od["daynum"] != 7)]
trips_od

In [ ]:
trips_od_rest = trips_od[(trips_od["origin_purpose"]=="Went to restaurant to eat/get take-out")
                            | (trips_od["dest_purpose"]=="Went to restaurant to eat/get take-out")]
trips_od_rest.to_csv('data_outputs/trips_od_rest.csv')

In [ ]:
# for each person, obtain their most frequent day num
max_day_per_person = {}
for person in trips_od_rest.person_dim_id.unique():
    person_trips = trips_od_rest[trips_od_rest.person_dim_id == person]
    max_day_per_person[person] = person_trips['daynum'].value_counts().idxmax()


# what % of each class is a frequent weekender?

In [ ]:
probabilities = pd.read_table("mplus_analysis/4_class_probabilities.dat", sep="\s+", header=None)
person_data = pd.read_csv("mplus_analysis/lca_data.csv")

In [ ]:
# claen data
person_prob = pd.concat([probabilities[34], person_data], axis=1)
person_prob = person_prob.rename(columns={34:"Class"})
# person_prob = person_prob.drop("Unnamed: 0", axis=1)
person_prob

In [ ]:
# count number of people with each maxdaynum in percentage
max_day_counts = Counter(max_day_per_person.values())
max_day_counts

In [ ]:
# create df of person based on persion_id, class, and max day num
person_class = person_prob[["person_id", "Class"]].copy(deep=True)

# add col based on max day num
person_class["max_day"] = person_class["person_id"].map(max_day_per_person)
person_class

In [ ]:
# for each class, whawt % is a weekender? weekend daynums = 5,6
print("Class 1:" )
# print(person_class[person_class.Class == 1].value_counts("max_day", normalize=True).sort_index(ascending=True))
# print sum of 5 & 6, percentage
print(person_class[person_class.Class == 1].value_counts("max_day", normalize=True).sort_index(ascending=True)[4:6].sum())

print("\nClass 2:" )
# print(person_class[person_class.Class == 2].value_counts("max_day", normalize=True).sort_index(ascending=True))
# print sum of 5 & 6, percentage
print(person_class[person_class.Class == 2].value_counts("max_day", normalize=True).sort_index(ascending=True)[4:6].sum())

print("\nClass 3:" )
# print(person_class[person_class.Class == 3].value_counts("max_day", normalize=True).sort_index(ascending=True))
# print sum of 5 & 6, percentage
print(person_class[person_class.Class == 3].value_counts("max_day", normalize=True).sort_index(ascending=True)[4:6].sum())

print("\nClass 4:" )
# print(person_class[person_class.Class == 4].value_counts("max_day", normalize=True).sort_index(ascending=True))
# print sum of 5 & 6, percentage
print(person_class[person_class.Class == 4].value_counts("max_day", normalize=True).sort_index(ascending=True)[4:6].sum())

# print size of each class
print("\nClass 1 size: ", person_class[person_class.Class == 1].shape[0])
print("Class 2 size: ", person_class[person_class.Class == 2].shape[0])
print("Class 3 size: ", person_class[person_class.Class == 3].shape[0])
print("Class 4 size: ", person_class[person_class.Class == 4].shape[0])



# Processing Yelp for restaurant-level information
- note: about 4000 entries are missing price, range or category

In [ ]:
rest_1 = pd.read_csv("data_outputs/trips_od_yelp_1.csv")
rest_2 = pd.read_csv("data_outputs/trips_od_yelp_2.csv")
rest_3 = pd.read_csv("data_outputs/trips_od_yelp_3.csv")
rest_4 = pd.read_csv("data_outputs/trips_od_yelp_4.csv")
rest_5 = pd.read_csv("data_outputs/trips_od_yelp_5.csv")

In [ ]:
rest_pois = pd.concat([rest_1, rest_2, rest_3, rest_4, rest_5])
rest_pois = rest_pois.drop(["Unnamed: 0.1", "Unnamed: 0"], axis = 1)

home_loc = home_loc[["household_id", "final_home_lat", "final_home_lng"]]
rest_pois = pd.merge(rest_pois, home_loc, on="household_id", how="left")

rest_pois

In [ ]:
# drop 2021 trips
rest_pois = rest_pois[rest_pois["survey_year"] != 2021]
rest_pois

In [ ]:
def yelp_extractor(row):
    try:
        yelp_dict = ast.literal_eval(row["restaurant_info"])

        name = None
        price = None
        rating = None
        categories = None
        # make sure dict has all four required categories
        if len(yelp_dict) > 0:
            tot_keys = list(yelp_dict.keys())
            if "name" in tot_keys:
                name = yelp_dict["name"]
            if "price" in tot_keys:
                price = yelp_dict["price"]
            if "rating" in tot_keys:
                rating = yelp_dict["rating"]
            if "categories" in tot_keys:
                categories = []
                all_cats = yelp_dict["categories"]
                for ea_cat in all_cats:
                    categories.append(ea_cat["title"])

        return pd.Series([name, price, rating, categories])
    except:
        print(row.name)
        return pd.Series([None, None, None, None])


In [ ]:
rest_pois.restaurant_info.isna().sum()/rest_pois.shape[0]

In [ ]:
# apply yelp extractor function to dataframe
rest_pois[["name", "price", "rating", "categories"]] = rest_pois.apply(yelp_extractor, axis=1)
# drop restaurants who have price, rating, or category of "None"
rest_pois = rest_pois.dropna(axis=0, subset = ['price', 'rating', 'categories'])

In [ ]:
# replace categories column in df with a single caetgory
# look up first value in categories, replace with category simple key
def simplify_food_cat(row):
    # simplified categories, from chat GPT
    category_simple = dict()
    category_simple["bakery"] = ["Kombucha",'Tea Rooms','Creperies','Chocolatiers & Shops','Coffee Roasteries','Cafes', "Acai Bowls", "Bagels", "Bakeries", "Bubble Tea", "Candy Stores", "Coffee & Tea", "Cupcakes", "Custom Cakes", "Desserts", "Donuts", "Gelato", "Ice Cream & Frozen Yogurt", "Macarons", "Pancakes", "Patisserie/Cake Shop", "Pretzels", "Shaved Ice", "Waffles"]
    category_simple["bars"]= ["Beer Tours", 'Cideries','Distilleries','Brewpubs','Bars', 'Beer Bar', 'Beer Gardens', 'Beer, Wine & Spirits', 'Breweries','Champagne Bars', 'Cocktail Bars', 'Dive Bars', 'Gay Bars', 'Gastropubs', 'Irish Pub', 'Karaoke', 'Lounges', 'Music Venues', 'Pubs', 'Speakeasies', 'Sports Bars', 'Whiskey Bars', 'Wine Bars', 'Wine Tasting Room', 'Wineries', 'Pool Halls', 'Venues & Event Spaces']
    category_simple["american"] = ["Fast Food",'Delis','Chicken Wings','Cajun/Creole', 'Barbeque',"American (New)", "American (Traditional)", "Burgers", "Breakfast & Brunch", "Cheesesteaks", "Comfort Food", "Diners", "Hot Dogs", "Sandwiches", "Soul Food", "Southern", "Steakhouses", "Hot Dogs", "New Mexican Cuisine", "Salad", "Wraps"]
    category_simple["asian"] = ["Sri Lankan",'Teppanyaki','Hawaiian','Poke', "Asian Fusion", "Cantonese", "Chinese", "Conveyor Belt Sushi", "Dim Sum", "Izakaya", "Japanese", "Japanese Curry", "Korean", "Laotian", "Malaysian", "Mongolian", "Noodles", "Pakistani", "Pan Asian", "Ramen", "Shanghainese", "Sushi Bars", "Taiwanese", "Thai", "Vietnamese", "Cambodian", "Indian", "Filipino", "Himalayan/Nepalese", "Szechuan", "Hot Pot"]
    category_simple["european"] = ['Sicilian','Fish & Chips',"British", "French", "German", "Italian", "Modern European", "Polish", "Russian", "Sardinian", "Scandinavian", "Scottish", "Spanish", "Tapas/Small Plates", "Turkish", "Tuscan", "Tapas Bars", "Irish", "Australian", "Pizza"]
    category_simple["hispanic"] = ["Argentine",'Tex-Mex',"Brazilian", "Cuban", "Empanadas", "Honduran", "Latin American", "Mexican", "Nicaraguan", "Peruvian", "Salvadoran", "Tacos", "Venezuelan", "Trinidadian"]
    category_simple["mediterranean"] = ['Halal','Falafel',"Greek", "Lebanese", "Mediterranean", "Middle Eastern", "Persian/Iranian", "Syrian", "Afghan"]
    category_simple["african"] = ['Moroccan',"African", "Egyptian", "Eritrean", "Ethiopian", "Somali", 'Caribbean']
    category_simple["specialty"] = ["Pop-Up Restaurants","Dinner Theater",'Honey','Chicken Shop','Caterers','Themed Cafes','Soup','Vegetarian','Butcher','Vegan','Juice Bars & Smoothies', 'Buffets','Cafeteria','Cheese Shops', 'Farmers Market', 'Fruits & Veggies', 'Gluten-Free', 'Health Markets', 'Imported Food', 'International Grocery', 'Meat Shops', 'Nutritionists', 'Organic Stores', 'Seafood Markets', 'Specialty Food', 'Vitamins & Supplements', 'Food Court', 'Food Delivery Services', 'Food Stands', 'Food Trucks', 'Herbal Shops', 'Herbs & Spices', 'Seafood', 'Street Vendors', 'Traditional Chinese Medicine', 'Imported Food', 'Local Flavor', 'Eatertainment', 'Food Tours']
    category_simple["misc"] = ["Casinos","Saunas","Yoga","Pop-up Shops","Furniture Stores","Restaurants","Hair Salons","CSA",'Pharmacy','Food','Grocery','Accessories', 'Arcades', 'Art Galleries', 'Art Museums', 'Auto Repair', 'Bikes', 'Boat Charters', 'Boating', 'Bocce Ball', 'Bookstores', 'Car Wash', 'Convenience Stores', 'Cooking Classes', 'Cooking Schools', 'Cosmetics & Beauty Supply', 'Department Stores', 'Drugstores', 'Florists', 'Gas Stations', 'Gift Shops', 'Hobby Shops', 'Home Decor', 'Mini Golf', 'Mobile Phones', 'Nurseries & Gardening', 'Photography Stores & Services', 'Propane', 'Sports Wear', 'Tobacco Shops', 'Toy Stores', 'Trainers', 'Vinyl Records']
    
    fast_food = False
    simple_cat = None
    if row.categories and len(row.categories) > 0:
        if "Fast Food" in row.categories:
            fast_food = True
            
        cat = row.categories[0]
        for key, ls in category_simple.items():
            if cat in ls:
                simple_cat = key
                break
    else:
        print('NoneType')
        
    return pd.Series([fast_food, simple_cat])

In [ ]:
# simplify and quantify food categories, count number of total transactions (including dine-in)
# count price, round ratings to nearest int,
rest_pois[["fast_food","simple_cat"]] = rest_pois.apply(simplify_food_cat, axis=1)
rest_pois['int_fast_food'] = pd.factorize(rest_pois["fast_food"], sort=True)[0] + 1
rest_pois['int_simple_cat'] = pd.factorize(rest_pois["simple_cat"], sort=True)[0] + 1 
rest_pois["int_price"] = rest_pois["price"].apply(lambda x: len(x))
rest_pois["int_rating"] = rest_pois["rating"].apply(lambda x: int(np.around(x)))

# Adding BE info for restaurant-level 

In [ ]:
# BE info for restaurants (o_tract10, d_tract10)
employ_pop = # download ACS data with # employed individuals per census tract
income = # download ACS data with median household income per census tract

In [ ]:
# clean up income columns, extract census tract numbers, drop rows with "-"
income_clean = income[["GEO_ID", "B19013_001E"]].tail(-1).reset_index(drop=True)
income_clean["census_tract"] = income_clean["GEO_ID"].apply(lambda x: x[9:])
income_clean

In [ ]:
# clean up employment population and convert tract to FIPS id
employ_pop = employ_pop[["data_year", "County", "Tract", "Total"]]
employ_pop = employ_pop[employ_pop.County != "Region"]
employ_pop = employ_pop[~np.isnan(employ_pop.Tract)]

# function to convert census tract numbers to FIPS codes
def tract_to_fips(row):
    if np.isnan(row.Tract):
        return
    # Define a dictionary of county FIPS codes for Washington
    all_county_fips = {'King': '033', 'Pierce': '053', 'Snohomish': '061', 'Kitsap': '035'}
    county_fips = all_county_fips[row.County]
    # Define the state FIPS code for Washington
    state_fips = '53'
    
    census_tract = str(int(float(row.Tract) * 100))
    fips_code = str(state_fips) + str(county_fips).zfill(3) + census_tract.zfill(6)
    
    return fips_code

employ_pop["tract_FIPS"] = employ_pop.apply(tract_to_fips, axis=1)
employ_pop["Total"] = employ_pop.Total.apply(lambda x: int(x) if x != "suppressed" else None)
employ_pop

In [ ]:
employ_pop = employ_pop[~np.isnan(employ_pop.Total)]
employ_pop_group = employ_pop.groupby("tract_FIPS").Total.median().reset_index().sort_values(by="Total")
employ_pop_group

In [ ]:
# drop any rows with nans in origin or destination tracts
rest_pois = rest_pois.dropna(axis=0, subset = ['o_tract10', 'd_tract10'])

In [ ]:
missed = 0 
def rest_be(row):
    global missed
    # determine if restaurant is origin or destination, find census tract
    if row.origin_purpose == "Went to restaurant to eat/get take-out":
        rest_census_tr = row.o_tract10
    elif row.dest_purpose == "Went to restaurant to eat/get take-out":
        rest_census_tr = row.d_tract10
    
    rest_census_tr = str(int(rest_census_tr))

    # look up census tract in pop and income dfs
    rest_empl_pop = None
    try:
        rest_empl_pop = employ_pop_group[employ_pop_group.tract_FIPS == rest_census_tr]["Total"].values[0]
    except IndexError:
        print("Not in index")
        missed += 1
    rest_inc = income_clean[income_clean.census_tract == rest_census_tr]["B19013_001E"].values[0]
    return pd.Series([rest_empl_pop, rest_inc])

In [ ]:
rest_pois[["rest_empl_pop", "rest_inc"]] = rest_pois.apply(rest_be, axis=1)
print(missed)

In [ ]:
# get rid of empl_pop == None (index didn't match up)
rest_pois = rest_pois[~rest_pois.rest_empl_pop.isna()]
rest_pois

In [ ]:
# convert income to int datatype
rest_pois.loc[rest_pois["rest_inc"]=='-', "rest_inc"] = np.nan
rest_pois.loc[rest_pois["rest_inc"]=='250,000+', "rest_inc"] = 250000
rest_pois["rest_inc"] = rest_pois["rest_inc"].apply(lambda x: float(x))

In [ ]:
# convenience-- distance from home
def dist_latlng(row):
    if row.origin_purpose == "Went to restaurant to eat/get take-out":
        coords_1 = (row.origin_lat, row.origin_lng)
    elif row.dest_purpose == "Went to restaurant to eat/get take-out":
        coords_1 = (row.dest_lat, row.dest_lng)
        
    coords_2 = (row.final_home_lat, row.final_home_lng)
    
    return geopy.distance.geodesic(coords_1, coords_2).miles

rest_pois["dist_from_home"] = rest_pois.apply(dist_latlng, axis=1)

# Combine restaurant with non-restaurant trips

In [ ]:
# ID non-restaurant trips
trips_od_nonrest = trips_od[(trips_od["origin_purpose"]!="Went to restaurant to eat/get take-out")
                            & (trips_od["dest_purpose"]!="Went to restaurant to eat/get take-out")]
trips_od_nonrest

In [ ]:
# combine restaurant and non-restaurant trips
trips_w_pois = pd.concat([rest_pois, trips_od_nonrest])
trips_w_pois

# Adding BE data for the person level
- restaurant in food desert, home in food desert?
- note: FARA (food desert) information is based off 2010 census tracts, and was updated in 2019. this data already has 2010 population and median family income information.
- Note: a food desert by USDA is defined as living more than 1 mile (in urban) or 20 miles (in rural) areas from a grocery store, and low income (LILA at 1 and 20)

In [ ]:
# drop 2021 data
household = household[household["survey_year"] != 2021]
person = person[person["survey_year"] != 2021]

In [ ]:
# read in USDA Food Access data
fara = # download USDA Food Access Research Atlas - 2019

In [ ]:
# only select census tracts located within PSRC
psrc_fara = fara[(fara.State == "Washington") & ((fara.County == "King County") |
                                  (fara.County == "Pierce County") | 
                                   (fara.County == "Snohomish County") |
                                  (fara.County == "Kitsap County"))]

In [ ]:
# need to join household to person data to obtain home tract
hh_info = household[["sample_county", "final_home_tract10", "household_id"]]
person = person.merge(hh_info, on = "household_id", how="left")
# only look at people who made restaurant trips
person_rest = person[person.person_id.isin(rest_person_ids)]

In [ ]:
# if a person lives in a home located in a food desert _-> home_fd = True, else = False
def in_fd(row):
    # tract of home
    census_tract = row.final_home_tract10 
    if psrc_fara[psrc_fara.CensusTract == census_tract]["LILATracts_1And10"].values[0] == 1:
        return True
    return False

person_rest["home_fd"] = person_rest.apply(in_fd, axis=1)

# Aggregate trips to the person level
- `trips_w_pois` is df with all trips made by people who made restaurant trips in the dataset
-`person_rest` is df with unique people who made restaurant trips
- characteristics from `trips_w_pois` should be aggregated for each unique person and added onto the `person_rest` df
    - travel time: median
    - mode: most common mode
    - trip purposes: most common trip purpose
    - time of day: most common time of day
    - day of week: most common day of week
    - ratio of restaurant trips to all trips: (not aggreggated)
    - food categories: most common, number of food categories visited
    - fast food: yes if visited in any of the trips, % fast food visits out of restaurant visits
    - distance from home: median
    - price: median
    - rating: median
    - population: median
    - income: median

In [ ]:
# converting time of day to mealtime
def time_meal(date_ls):
    meal_ls = []
    for date in date_ls:
        date_string = date[:-1]
        date_format = "Date: %Y-%m-%d %H:%M:%S.%f"
        five_am = datetime.strptime("05:00", "%H:%M").time()
        ten_am = datetime.strptime("10:00", "%H:%M").time()
        two_pm = datetime.strptime("14:00", "%H:%M").time()
        five_pm = datetime.strptime("17:00", "%H:%M").time()
        nine_pm = datetime.strptime("21:00", "%H:%M").time()
        tod = datetime.strptime(date_string, date_format).time()
        if tod > five_am and tod < ten_am:
            meal_ls.append("Breakfast")
        elif tod > ten_am and tod < two_pm:
            meal_ls.append("Lunch")
        elif tod > five_pm and tod < nine_pm:
            meal_ls.append("Dinner")
        else:
            meal_ls.append("Other")
    return max(set(meal_ls), key=meal_ls.count)

meal_pref = rest_pois.groupby("person_dim_id")["depart_time_string"].apply(list).reset_index()
meal_pref["meal"] = meal_pref.depart_time_string.apply(time_meal)
meal_pref

In [ ]:
# NOTE: THIS SHOULD ONLY BE RUN ON RESTAURANT TRIPS
# travel time
med_travel_time = rest_pois.groupby("person_dim_id")[["travel_time_calc"]].median()
med_travel_time = med_travel_time.rename({"travel_time_calc":"med_trav_time"}, axis=1)

# mode (note: don't want nan)
all_modes = rest_pois.groupby("person_dim_id")["main_mode"].apply(list).reset_index()
all_modes["pref_modes"] = all_modes["main_mode"].apply(lambda x:Counter(x).most_common(1)[0][0])

# trip ratios
rest_trips = rest_pois.groupby("person_dim_id").trip_id.count().reset_index().rename({"trip_id":"rest_trips"}, axis=1)
non_rest_trips = trips_od_nonrest.groupby("person_dim_id").trip_id.count().reset_index().rename({"trip_id":"nonrest_trips"}, axis=1)
trip_ratios = pd.merge(rest_trips, non_rest_trips, on="person_dim_id", how="left")
trip_ratios["rest:non_rest"] = trip_ratios["rest_trips"]/trip_ratios["nonrest_trips"]

# food categories
all_food_cat = rest_pois.groupby("person_dim_id")["simple_cat"].apply(list).reset_index()
all_food_cat["diversity_food_cat"] = all_food_cat["simple_cat"].apply(lambda x: len(set(x)))
all_food_cat["pref_food_cat"] = all_food_cat["simple_cat"].apply(lambda x: Counter(x).most_common(1)[0][0])

# fast food
fast_food = rest_pois.groupby("person_dim_id")["fast_food"].sum().reset_index()
fast_food = pd.merge(rest_trips, fast_food, on="person_dim_id")
fast_food["fast_food_flag"] = fast_food["fast_food"].apply(lambda x: True if x>0 else False)
fast_food["pct_fast_food"] = fast_food["fast_food"]/fast_food["rest_trips"]

# price
price_avg = rest_pois.groupby("person_dim_id")[["int_price"]].median()
price_avg = price_avg.rename({"int_price":"med_price"}, axis=1)

# rating
rating_avg = rest_pois.groupby("person_dim_id")[["rating"]].median()
rating_avg = rating_avg.rename({"rating":"med_rating"}, axis=1)

#population
rest_empl_avg = rest_pois.groupby("person_dim_id")[["rest_empl_pop"]].median()
rest_empl_avg = rest_empl_avg.rename({"Total":"med_rest_empl"}, axis=1)

# income
rest_inc_avg = rest_pois.groupby("person_dim_id")[["rest_inc"]].median()
rest_inc_avg = rest_inc_avg.rename({"rest_inc":"med_rest_inc"}, axis=1)

# distance from home
dist_home_avg = rest_pois.groupby("person_dim_id")[["dist_from_home"]].median()

In [ ]:
# trip purpose (note: don't want missing)
def nonrest_purp(row):
    purp = None
    if row.origin_purpose_cat == "Meal":
        purp = row.dest_purpose_cat
    elif row.dest_purpose_cat == "Meal":
        purp = row.origin_purpose_cat
    return purp

rest_pois["nonrest_purp"] = rest_pois.apply(nonrest_purp, axis=1)

trip_purp = rest_pois.groupby("person_dim_id")["nonrest_purp"].apply(list).reset_index()
trip_purp["nonrest_purp"] = trip_purp["nonrest_purp"].apply(lambda x: Counter(x).most_common(1)[0][0])

In [ ]:
person_agg = pd.merge(person_rest, med_travel_time, right_on="person_dim_id", left_on="person_id", how='left')
person_agg = pd.merge(person_agg, all_modes[["person_dim_id","pref_modes"]], left_on = "person_id", right_on = "person_dim_id", how='left')
person_agg = person_agg.drop(["person_dim_id"], axis=1)
person_agg = pd.merge(person_agg, all_food_cat[["person_dim_id","diversity_food_cat"]], left_on = "person_id", right_on = "person_dim_id", how='left')
person_agg = person_agg.drop(["person_dim_id"], axis=1)
person_agg = pd.merge(person_agg, fast_food[["person_dim_id", "pct_fast_food"]], left_on = "person_id", right_on = "person_dim_id", how='left')
person_agg = person_agg.drop(["person_dim_id"], axis=1)
person_agg = pd.merge(person_agg, price_avg, right_on="person_dim_id", left_on="person_id", how='left')
person_agg = pd.merge(person_agg, rating_avg, right_on="person_dim_id", left_on="person_id", how='left')
person_agg = pd.merge(person_agg, rest_empl_avg, right_on="person_dim_id", left_on="person_id", how='left')
person_agg = pd.merge(person_agg, rest_inc_avg, right_on="person_dim_id", left_on="person_id", how='left')
person_agg = pd.merge(person_agg, dist_home_avg, right_on="person_dim_id", left_on="person_id", how='left')
person_agg = pd.merge(person_agg, trip_ratios[["person_dim_id","rest:non_rest"]], right_on="person_dim_id", left_on="person_id", how='left')
person_agg = person_agg.drop(["person_dim_id"], axis=1)
person_agg = pd.merge(person_agg, trip_purp, left_on = "person_id", right_on = "person_dim_id", how='left')
person_agg = person_agg.drop(["person_dim_id"], axis=1)
person_agg = pd.merge(person_agg, meal_pref[["person_dim_id","meal"]], left_on = "person_id", right_on = "person_dim_id", how='left')
person_agg = person_agg.drop(["person_dim_id"], axis=1)
person_agg



# Clean up data for analysis
- drop columns
- transform categorical --> numerical

In [ ]:
for col in person_agg.columns:
    print(col)

In [ ]:
vars_keep = ["household_id", "person_id", "survey_year", "seattle_home",
               "sample_county", "hhsize", "vehicle_count", "hhincome_broad", "hhincome_detailed", 
               "rent_own", "numchildren", "age", "gender", "employment", "student", "education",
               "license", "race_category", "hh_race_category", "commute_freq", 
                "prev_res_factors_housing_cost",
                "prev_res_factors_income_change", "prev_res_factors_community_chan",
                "prev_res_factors_hh_size", "prev_res_factors_more_space",
                "prev_res_factors_less_space", "prev_res_factors_employment",
                "prev_res_factors_school", "prev_res_factors_crime", "prev_res_factors_quality",
                "prev_res_factors_forced", "prev_res_factors_no_answer", "prev_res_factors_other",
                "prev_res_factors_specify", "res_factors_30min", "res_factors_afford",
                "res_factors_closefam", "res_factors_hhchange", "res_factors_hwy",
                "res_factors_school", "res_factors_space", "res_factors_transit",
                "res_factors_walk", "home_fd", "med_trav_time", "pref_modes", "diversity_food_cat",
            "pct_fast_food", "med_price", "med_rating", "rest_empl_pop", "med_rest_inc",
            "dist_from_home","rest:non_rest", "nonrest_purp", "meal"]
final_person_data = person_agg[vars_keep]

In [ ]:
# save as csv
final_person_data.to_csv("data_outputs/final_person_data.csv")